**1: Funções Auxiliares**

In [1]:
import numpy as np # type: ignore
import random

def is_palindrome(sequence):
    """Verifica se uma sequência é um palíndromo."""
    return sequence == sequence[::-1]

def generate_palindromes():
    """
    Gera todos os palíndromos de 3 dígitos possíveis, excluindo aqueles com três números iguais.
    """
    palindromes = []
    for i in range(1, 10):
        for j in range(1, 10):
            if i != j:  # Exclui palíndromos do tipo 111, 222, ...
                palindromes.append([i, j, i])
    random.shuffle(palindromes)  # Embaralha para introduzir variação
    return palindromes

def is_valid_palindrome(palindrome, used_palindromes):
    """Verifica se o palíndromo não foi usado em outros subquadrantes."""
    return palindrome not in used_palindromes

def can_add_to_subquadrant(subquadrant, num, palindrome, duplicate_counts):
    """
    Verifica se um número pode ser inserido no subquadrante.
    """
    # Regra 1: Não pode estar no palíndromo
    if num in palindrome:
        return False

    # Regra 2: O número pode aparecer no máximo 2 vezes
    count = np.count_nonzero(subquadrant == num)
    if count >= 2:
        return False

    # Regra 3: Apenas dois números extras podem ser duplicados
    if count == 1 and len(duplicate_counts) >= 2 and num not in duplicate_counts:
        return False

    return True


**2: Backtracking para Palíndromos**

In [2]:
def fill_palindromes(board):
    """
    Preenche as diagonais principais dos subquadrantes 3x3 com palíndromos.
    Retorna True se for possível preencher os palíndromos.
    """
    palindromes = generate_palindromes()  # Embaralha a cada execução
    used_palindromes = []
    
    for sq_row in range(3):  # Para cada linha de subquadrantes
        for sq_col in range(3):  # Para cada coluna de subquadrantes
            found = False
            for palindrome in palindromes:
                if is_valid_palindrome(palindrome, used_palindromes):
                    # Preencher diagonal principal
                    board[sq_row*3 + 0][sq_col*3 + 0] = palindrome[0]
                    board[sq_row*3 + 1][sq_col*3 + 1] = palindrome[1]
                    board[sq_row*3 + 2][sq_col*3 + 2] = palindrome[2]
                    
                    used_palindromes.append(palindrome)
                    found = True
                    break
            if not found:
                return False  # Não foi possível preencher os palíndromos
    
    return True


**3: Backtracking para o Restante do Tabuleiro**

In [3]:
def fill_subquadrants(board):
    """
    Preenche o restante das células em cada subquadrante.
    """
    for sq_row in range(3):
        for sq_col in range(3):
            # Coordenadas do subquadrante
            start_row, start_col = sq_row * 3, sq_col * 3
            subquadrant = board[start_row:start_row+3, start_col:start_col+3]
            palindrome = [
                subquadrant[0, 0],
                subquadrant[1, 1],
                subquadrant[2, 2]
            ]
            
            # Coletar números extras duplicados no subquadrante
            duplicate_counts = {
                num for num in subquadrant.flatten()
                if np.count_nonzero(subquadrant == num) > 1 and num not in palindrome
            }
            
            for row in range(3):
                for col in range(3):
                    if subquadrant[row, col] == 0:  # Célula vazia
                        nums = list(range(1, 10))
                        random.shuffle(nums)  # Embaralha os números possíveis
                        for num in nums:
                            if can_add_to_subquadrant(subquadrant, num, palindrome, duplicate_counts):
                                subquadrant[row, col] = num
                                if fill_subquadrants(board):
                                    return True
                                subquadrant[row, col] = 0  # Undo move
                        return False  # Nenhum número válido encontrado
    return True


**Célula 4: Geração de Tabuleiros**

In [4]:
def generate_boards(n):
    """
    Gera os primeiros n tabuleiros que atendem às restrições.
    """
    boards = []
    attempts = 0  # Número de tentativas para evitar loops infinitos
    
    while len(boards) < n and attempts < 1000:
        board = np.zeros((9, 9), dtype=int)
        if fill_palindromes(board) and fill_subquadrants(board):
            # Verifica se o tabuleiro gerado já existe (evita duplicatas)
            if not any(np.array_equal(board, b) for b in boards):
                boards.append(board.copy())
        attempts += 1
    
    return boards

# Função para exibir o tabuleiro
def print_board(board):
    for i in range(9):
        if i % 3 == 0 and i != 0:
            print("-" * 21)
        row = ""
        for j in range(9):
            if j % 3 == 0 and j != 0:
                row += " | "
            row += f"{board[i][j]} "
        print(row)

# Geração e exibição dos tabuleiros
n = 2
boards = generate_boards(n)

for i, board in enumerate(boards, start=1):
    print(f"\nTabuleiro {i}:")
    print_board(board)


Tabuleiro 1:
2 7 8  | 7 5 5  | 6 4 3 
4 6 8  | 6 2 1  | 7 1 4 
9 1 2  | 4 3 7  | 2 7 6 
---------------------
2 4 6  | 1 3 6  | 7 5 3 
5 8 1  | 6 5 4  | 9 4 9 
6 4 2  | 8 4 1  | 1 2 7 
---------------------
1 9 4  | 4 8 8  | 1 7 4 
6 2 7  | 6 2 7  | 9 6 4 
7 9 1  | 5 7 4  | 2 3 1 

Tabuleiro 2:
8 7 3  | 9 1 1  | 4 3 3 
4 1 2  | 4 3 6  | 6 7 9 
7 3 8  | 7 7 9  | 8 2 4 
---------------------
9 7 4  | 3 4 9  | 5 9 7 
4 8 1  | 2 7 1  | 8 3 2 
3 3 9  | 1 8 3  | 9 6 5 
---------------------
4 8 6  | 7 6 9  | 8 2 3 
5 9 1  | 6 8 1  | 3 7 5 
5 1 4  | 5 5 7  | 4 4 8 
